# Box Office Movie Franchise Predictor - Modeling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

import plotly.express as px
import plotly.graph_objects as go

%config InlineBackend.figure_format='retina'

## Load Data File(s)

In [2]:
data_path = './data/'

file_all = 'movies_with_sequels_final_cleaned.csv'
file_org = 'movies_with_originals_only_final_cleaned.csv'
file_seq = 'movies_with_sequels_only_final_cleaned.csv'

df_all = pd.read_csv(data_path + file_all)
df_org = pd.read_csv(data_path + file_org)
df_seq = pd.read_csv(data_path + file_seq)

In [3]:
df_all['Release Date'] = pd.to_datetime(df_all['Release Date'])
df_org['Release Date'] = pd.to_datetime(df_org['Release Date'])
df_seq['Release Date'] = pd.to_datetime(df_seq['Release Date'])

In [4]:
# Get 
def diff_rel_dates(data_org,data_seq,delta_type):
    
    delta_time = (data_seq['Release Date'] - data_org['Release Date'])/np.timedelta64(1, delta_type)
    
    neg_delta = delta_time[delta_time < 0]
    
    if neg_delta is not None:
        print('No negative difference')
    else:
        print('There are negative time(s) between sequel and original. Please check!')

    return delta_time

In [5]:
# Set years as the difference between sequel and original
diff_type = 'Y'

diff_time = diff_rel_dates(df_org,df_seq,diff_type)

No negative difference


In [6]:
df_org.head()

,Title,url,IMDB Score,Metacritic,Runtime (mins),Budget,Opening Weekend,Gross USA,Gross World,Release Date,Rating,Genres,Country
0,Spider-Man (2002),http://www.imdb.com/title/tt0145487/,7.3,73.0,121,139000000.0,114844116.0,407022860.0,8.250250e+08,2002-05-03,PG-13,Action Adventure Sci-Fi,USA
1,The Matrix (1999),http://www.imdb.com/title/tt0133093/,8.7,73.0,136,63000000.0,27788331.0,171479930.0,4.653438e+08,1999-03-31,R,Action Sci-Fi,USA
2,The Lord of the Rings: The Fellowship of the R...,http://www.imdb.com/title/tt0120737/,8.8,92.0,178,93000000.0,47211490.0,315544750.0,8.878328e+08,2001-12-19,PG-13,Action Adventure Drama,New Zealand
3,The Lord of the Rings: The Return of the King ...,http://www.imdb.com/title/tt0167260/,8.9,94.0,201,94000000.0,72629713.0,377845905.0,1.142219e+09,2003-12-17,PG-13,Adventure Drama Fantasy,New Zealand
4,The Hobbit: The Desolation of Smaug (2013),http://www.imdb.com/title/tt1170358/,7.8,66.0,161,225000000.0,73645197.0,258366855.0,9.583669e+08,2013-12-13,PG-13,Adventure Fantasy,New Zealand


In [7]:
df_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 539 entries, 0 to 538
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Title            539 non-null    object        
 1   url              539 non-null    object        
 2   IMDB Score       539 non-null    float64       
 3   Metacritic       505 non-null    float64       
 4   Runtime (mins)   539 non-null    int64         
 5   Budget           539 non-null    float64       
 6   Opening Weekend  475 non-null    float64       
 7   Gross USA        508 non-null    float64       
 8   Gross World      539 non-null    float64       
 9   Release Date     539 non-null    datetime64[ns]
 10  Rating           539 non-null    object        
 11  Genres           539 non-null    object        
 12  Country          539 non-null    object        
dtypes: datetime64[ns](1), float64(6), int64(1), object(5)
memory usage: 54.9+ KB
